<a href="https://colab.research.google.com/github/sushirito/Molecular-Dynamics/blob/carbon-structures/Copy_of_Mercury_Graphene_Sheet_MD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies and compile LAMMPS


In [ ]:
# Install dependencies and compile LAMMPS
%%capture
!apt-get update
!apt-get install -y build-essential cmake libfftw3-dev libjpeg-dev libpng-dev \
                    libopenmpi-dev openmpi-bin python3-dev python3-numpy git
# Clone the LAMMPS repository
%cd /content
!git clone -b stable https://github.com/lammps/lammps.git
%cd lammps

# Create a build directory and compile LAMMPS with required packages
!mkdir build
%cd build
!cmake ../cmake -DBUILD_SHARED_LIBS=yes \
                -DLAMMPS_EXCEPTIONS=yes \
                -DPKG_MOLECULE=yes \
                -DPKG_KSPACE=yes \
                -DPKG_RIGID=yes \
                -DPKG_MANYBODY=yes \
                -DPKG_USER-MISC=yes \
                -DPKG_PYTHON=yes \
                -DPYTHON_EXECUTABLE=`which python3`
!make -j4
!make install-python

# Return to the working directory
%cd /content/

In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "4"

In [ ]:
!pip install MDAnalysis

import numpy as np
import MDAnalysis as mda
from MDAnalysis.lib.util import openany
from MDAnalysis.analysis.rdf import InterRDF
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
from scipy.optimize import curve_fit
from scipy.signal import find_peaks
import math, sys, os

np.random.seed(42)

In [ ]:
!wget -O CH.airebo https://raw.githubusercontent.com/simongravelle/lammps-input-files/main/inputs/amorphous-carbon/CH.airebo

--2025-01-12 22:56:27--  https://raw.githubusercontent.com/simongravelle/lammps-input-files/main/inputs/amorphous-carbon/CH.airebo
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 926693 (905K) [text/plain]
Saving to: ‘CH.airebo’

CH.airebo           100%[===================>] 904.97K  --.-KB/s    in 0.04s   

2025-01-12 22:56:27 (21.9 MB/s) - ‘CH.airebo’ saved [926693/926693]



In [ ]:
# Save the activated carbon script
with open("activated_carbon.in", "w") as f:
    f.write("""
# This LAMMPS input script simulates the formation of amorphous carbon structure
# Units and setup
units metal
atom_style full
boundary p p p
pair_style airebo 2.5 1 1

# Create box and atoms
region box block -20 20 -20 20 -20 20
create_box 1 box
create_atoms 1 random 5000 45287 box

# Force field
mass 1 12.0107
pair_coeff * * CH.airebo C

# Energy minimization
thermo 10
minimize 1.0e-4 1.0e-6 100 1000
reset_timestep 0

# Initial velocities and NVT fix
velocity all create 10000 49284 mom yes rot yes dist gaussian
fix mynvt all nvt temp 10000 4000 0.1
timestep 0.0005

# Dumps and thermo
dump mydump all atom 10000 dump.lammpstrj
thermo 1000

# First run: Decrease temperature
run 100000

# Further steps
fix mynvt all nvt temp 4000 4000 0.1
run 500000

fix mynvt all nvt temp 4000 3000 0.1
run 500000

fix mynvt all nvt temp 3000 300 0.1
run 100000

fix mynvt all nvt temp 300 300 0.1
run 10000

# Save final state
write_data final_state.data
    """)

In [ ]:
# Run the simulation
! /content/lammps/build/lmp -in activated_carbon.in

LAMMPS (29 Aug 2024 - Update 1)
  using 4 OpenMP thread(s) per MPI task
Created orthogonal box = (-20 -20 -20) to (20 20 20)
  1 by 1 by 1 MPI processor grid
Created 5000 atoms
  using lattice units in orthogonal box = (-20 -20 -20) to (20 20 20)
  create_atoms CPU = 0.002 seconds
Reading airebo potential file CH.airebo with DATE: 2011-10-25
Neighbor list info ...
  update: every = 1 steps, delay = 0 steps, check = yes
  max neighbors/atom: 2000, page size: 100000
  master list distance cutoff = 10.5
  ghost atom cutoff = 10.5
  binsize = 5.25, bins = 8 8 8
  1 neighbor lists, perpetual/occasional/extra = 1 0 0
  (1) pair airebo, perpetual
      attributes: full, newton on, ghost
      pair build: full/bin/ghost
      stencil: full/ghost/bin/3d
      bin: standard
Setting up cg style minimization ...
  Unit style    : metal
  Current step  : 0
Per MPI rank memory allocation (min/avg/max) = 28.97 | 28.97 | 28.97 Mbytes
   Step          Temp          E_pair         E_mol          TotEng 